In [12]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv
import ssl

sys.path.append('..')
sys.path.append('../..')
sys.path.append('../../..')
ssl._create_default_https_context = ssl._create_stdlib_context
load_dotenv()

# Must restart the kernel if any of the following imported modules are changed
from helpers.focusmate import fm_sessions_data_to_df
from api.helpers.focusmate import fetch_all_focusmate_sessions, fetch_focusmate_profile
from api.helpers.request import get_access_token_from_db
from api.test.sample_data.sessions import sample_sessions

fm_api_profile_endpoint = os.getenv("NEXT_PUBLIC_FM_API_PROFILE_ENDPOINT")
fm_api_sessions_endpoint = os.getenv("NEXT_PUBLIC_FM_API_SESSIONS_ENDPOINT")
session_id = ""
local_timezone = "America/New_York"

In [13]:
access_token = get_access_token_from_db(session_id)

profile_data = fetch_focusmate_profile(
        fm_api_profile_endpoint, access_token).get("user")
local_timezone: str = profile_data.get("timeZone")

# Option A: use all sessions
member_since: str = profile_data.get("memberSince")
sessions_data = await fetch_all_focusmate_sessions(
    fm_api_sessions_endpoint, access_token, member_since)

# Option B: use sample sessions
# sessions_data = sample_sessions

all_sessions = fm_sessions_data_to_df(sessions_data, local_timezone)
sessions = all_sessions[all_sessions['completed'] == True].copy()

sessions_copy = sessions.copy()

# sessions_copy['start_date'] = sessions_copy['start_time'].dt.date

# sessions_copy.sort_values('start_date', inplace=True)

# sessions_copy.head()

In [21]:
sessions_copy['start_date'] = sessions_copy['start_time'].dt.strftime('%Y-%m-%d')
sessions_per_date = sessions_copy.groupby('start_date').size()
sessions_per_date = sessions_per_date.reset_index()
sessions_per_date.columns = ['date', 'value']
sessions_per_date = sessions_per_date.to_dict(orient='records')
sessions_per_date

[{'date': '2024-05-28', 'value': 1},
 {'date': '2024-05-31', 'value': 2},
 {'date': '2024-06-01', 'value': 1},
 {'date': '2024-06-03', 'value': 1},
 {'date': '2024-06-04', 'value': 1},
 {'date': '2024-06-05', 'value': 1},
 {'date': '2024-06-07', 'value': 1}]